In [1]:
import pandas as pd
import sqlite3
import os
import json
import time
import requests

In [37]:
conn = sqlite3.connect('data.sqlite3')

In [39]:
uf_df = pd.read_sql_query('SELECT * FROM users_faculties', con=conn)

In [40]:
us_df = pd.read_sql_query('SELECT * FROM users_subscriptions', con=conn)

In [50]:
uf_df

,user_id,faculty_id,graduation,is_closed
0,9341862,2470,1993.0,0
1,166277444,2470,2009.0,0
2,4098790,2470,2008.0,0
3,4098790,2139043,2008.0,0
4,237165798,2470,2009.0,0
...,...,...,...,...
10174,552810176,2223186,2020.0,0
10175,545950940,None,NaN,0
10176,545950940,2266966,NaN,0
10177,520446033,2266966,2005.0,0


In [52]:
fac_df = pd.read_sql_query('SELECT * FROM faculties', con=conn)

In [56]:
facuser = fac_df.merge(uf_df, how='left', on='faculty_id')

In [57]:
facuser.groupby('faculty_id')['user_id'].nunique()

faculty_id
1034731    234
106511     268
203338     373
20422      356
2139043    275
2139044    173
2139051    329
2139211    258
2161061    286
2161062    357
2161063     68
2173844    343
2174103    120
2218244    352
2223186    364
2231267    371
2231268    185
2231275    385
2231276     77
2231283     31
2231284     81
2231285     34
2231790    378
2266965     73
2266966    267
2470       314
2951       261
2957       275
2959       325
2961       230
2973       390
40378      273
40773      207
623655     242
Name: user_id, dtype: int64

In [58]:
from sklearn import preprocessing

In [61]:
import numpy as np

In [79]:
ohe = preprocessing.OneHotEncoder() # categories=us_df['subs_id'], dtype=np.int32

In [80]:
result = ohe.fit_transform(us_df[['user_id', 'subs_id']])

In [86]:
ohe.get_params()

{'categories': 'auto',
 'drop': None,
 'dtype': numpy.float64,
 'handle_unknown': 'error',
 'sparse': True}

In [93]:
allin = facuser.merge(us_df[['user_id', 'subs_id']], on='user_id', how='inner')

In [94]:
allin

,faculty_id,title,user_id,graduation,is_closed,subs_id
0,2470,Механико-машиностроительный институт,9341862,1993.0,0,29184795
1,2470,Механико-машиностроительный институт,9341862,1993.0,0,26307864
2,2470,Механико-машиностроительный институт,9341862,1993.0,0,34137285
3,2470,Механико-машиностроительный институт,9341862,1993.0,0,33628480
4,2470,Механико-машиностроительный институт,9341862,1993.0,0,44674375
...,...,...,...,...,...,...
964495,2266966,Институт экономики и управления,448949782,NaN,0,35068738
964496,2266966,Институт экономики и управления,448949782,NaN,0,35486626
964497,2266966,Институт экономики и управления,448949782,NaN,0,35555555
964498,2266966,Институт экономики и управления,448949782,NaN,0,35806721


In [95]:
us_df['user_id'].nunique()

8081

In [99]:
facuser[~facuser.duplicated()]

,faculty_id,title,user_id,graduation,is_closed
0,2470,Механико-машиностроительный институт,9341862,1993.0,0
1,2470,Механико-машиностроительный институт,166277444,2009.0,0
2,2470,Механико-машиностроительный институт,4098790,2008.0,0
3,2470,Механико-машиностроительный институт,237165798,2009.0,0
4,2470,Механико-машиностроительный институт,233735155,NaN,0
...,...,...,...,...,...
9639,2266966,Институт экономики и управления,562122999,2002.0,0
9640,2266966,Институт экономики и управления,560783172,2016.0,0
9642,2266966,Институт экономики и управления,545950940,NaN,0
9643,2266966,Институт экономики и управления,520446033,2005.0,0
